In [1]:
# import dependencies
import pandas as pd

For testing purposes, I have been converting my app.py query to all left joins:

def get_listings():

    # hosts has nulls, so LEFT JOIN

    query = text(f"""SELECT * FROM listings

LEFT JOIN listing_description ON listings.listing_id = listing_description.id

LEFT JOIN listing_reviews ON listings.listing_id = listing_reviews.listing_id

LEFT JOIN hosts ON listings.listing_id = hosts.listing_id""")


But now I"m abandoning this scratchpad, and starting with schema 2.0

It works with the initial listings, but I'm going to skip recreating the rest of the initial tables. I'm tossing it all and starting over.

In [2]:
# read in listings data
df = pd.read_csv("../raw_data/listings_2024_mar.csv")
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3686,https://www.airbnb.com/rooms/3686,20230913045152,2023-09-13,city scrape,Home in Washington · ★4.64 · 1 bedroom · 1 bed...,IMPORTANT NOTES<br />* Carefully read and be s...,We love that our neighborhood is up and coming...,https://a0.muscache.com/pictures/61e02c7e-3d66...,4645,...,4.84,3.89,4.63,NaN,f,1,0,1,0,0.53
1,3943,https://www.airbnb.com/rooms/3943,20230913045152,2023-09-13,city scrape,Townhouse in Washington · ★4.83 · 1 bedroom · ...,You will be staying in high ceiling bedroom w...,This rowhouse is centrally located in the hear...,https://a0.muscache.com/pictures/airflow/Hosti...,5059,...,4.91,4.57,4.75,Hosted License: 5007242201001033,f,5,0,5,0,2.75
2,4197,https://www.airbnb.com/rooms/4197,20230913045152,2023-09-13,city scrape,Home in Washington · ★4.85 · 1 bedroom · 1 bed...,This is the middle bedroom upstairs in a resto...,"Our area, the Eastern Market neighborhood of C...",https://a0.muscache.com/pictures/miso/Hosting-...,5061,...,4.98,4.96,4.94,Hosted License: 5007242201000749,f,2,0,2,0,0.32
3,4529,https://www.airbnb.com/rooms/4529,20230913045152,2023-09-13,city scrape,Home in Washington · ★4.66 · 1 bedroom · 1 bed...,This is large private bedroom with plenty of...,Very quiet neighborhood and it is easy accessi...,https://a0.muscache.com/pictures/86072003/6709...,5803,...,4.93,4.51,4.83,Exempt,f,1,0,1,0,0.59
4,4967,https://www.airbnb.com/rooms/4967,20230913045152,2023-09-13,previous scrape,Home in Washington · ★4.74 · 1 bedroom · 1 bed...,"<b>The space</b><br />Hello, my name is Seveer...",NaN,https://a0.muscache.com/pictures/2439810/bb320...,7086,...,4.93,4.21,4.64,NaN,f,3,0,3,0,0.20


In [3]:
len(df)

6705

In [4]:
# print header names
print(df.columns)

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

<h2>Build initial listings table<h2>

In [5]:
# build initial listings df
listings_df = df[
    [
        "id",
        "name",
        "description",
        "listing_url",
        "neighborhood_overview",
        "picture_url",
        "neighbourhood",
        "neighbourhood_cleansed",
        "latitude",
        "longitude",
        "property_type",
        "room_type",
        "accommodates",
        "bathrooms_text",
        "bedrooms",
        "beds",
        "price",
        "license",
    ]
]
listings_df.head()

,id,name,description,listing_url,neighborhood_overview,picture_url,neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,price,license
0,3686,Home in Washington · ★4.64 · 1 bedroom · 1 bed...,IMPORTANT NOTES<br />* Carefully read and be s...,https://www.airbnb.com/rooms/3686,We love that our neighborhood is up and coming...,https://a0.muscache.com/pictures/61e02c7e-3d66...,"Washington, District of Columbia, United States",Historic Anacostia,38.86339,-76.98889,Private room in home,Private room,1,1 private bath,NaN,1.0,$67.00,NaN
1,3943,Townhouse in Washington · ★4.83 · 1 bedroom · ...,You will be staying in high ceiling bedroom w...,https://www.airbnb.com/rooms/3943,This rowhouse is centrally located in the hear...,https://a0.muscache.com/pictures/airflow/Hosti...,"Washington, District of Columbia, United States","Edgewood, Bloomingdale, Truxton Circle, Eckington",38.91195,-77.00456,Private room in townhouse,Private room,2,1 private bath,NaN,1.0,$78.00,Hosted License: 5007242201001033
2,4197,Home in Washington · ★4.85 · 1 bedroom · 1 bed...,This is the middle bedroom upstairs in a resto...,https://www.airbnb.com/rooms/4197,"Our area, the Eastern Market neighborhood of C...",https://a0.muscache.com/pictures/miso/Hosting-...,"Washington, District of Columbia, United States","Capitol Hill, Lincoln Park",38.88719,-76.99472,Private room in home,Private room,1,1.5 shared baths,NaN,1.0,$80.00,Hosted License: 5007242201000749
3,4529,Home in Washington · ★4.66 · 1 bedroom · 1 bed...,This is large private bedroom with plenty of...,https://www.airbnb.com/rooms/4529,Very quiet neighborhood and it is easy accessi...,https://a0.muscache.com/pictures/86072003/6709...,"Washington, District of Columbia, United States","Eastland Gardens, Kenilworth",38.90585,-76.94469,Private room in home,Private room,2,1 shared bath,NaN,1.0,$56.00,Exempt
4,4967,Home in Washington · ★4.74 · 1 bedroom · 1 bed...,"<b>The space</b><br />Hello, my name is Seveer...",https://www.airbnb.com/rooms/4967,NaN,https://a0.muscache.com/pictures/2439810/bb320...,NaN,"Ivy City, Arboretum, Trinidad, Carver Langston",38.91217,-76.99249,Private room in home,Private room,1,3 baths,NaN,1.0,"$2,500.00",NaN


In [6]:
len(listings_df)

6705

In [7]:
len(df)

6705

In [8]:
# rename id column to listing_id
listings_df.rename(columns={"id": "listing_id"}, inplace=True)
listings_df.head()

C:\Users\johbr\AppData\Local\Temp\ipykernel_12244\1181363841.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_df.rename(columns={"id": "listing_id"}, inplace=True)


,listing_id,name,description,listing_url,neighborhood_overview,picture_url,neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,price,license
0,3686,Home in Washington · ★4.64 · 1 bedroom · 1 bed...,IMPORTANT NOTES<br />* Carefully read and be s...,https://www.airbnb.com/rooms/3686,We love that our neighborhood is up and coming...,https://a0.muscache.com/pictures/61e02c7e-3d66...,"Washington, District of Columbia, United States",Historic Anacostia,38.86339,-76.98889,Private room in home,Private room,1,1 private bath,NaN,1.0,$67.00,NaN
1,3943,Townhouse in Washington · ★4.83 · 1 bedroom · ...,You will be staying in high ceiling bedroom w...,https://www.airbnb.com/rooms/3943,This rowhouse is centrally located in the hear...,https://a0.muscache.com/pictures/airflow/Hosti...,"Washington, District of Columbia, United States","Edgewood, Bloomingdale, Truxton Circle, Eckington",38.91195,-77.00456,Private room in townhouse,Private room,2,1 private bath,NaN,1.0,$78.00,Hosted License: 5007242201001033
2,4197,Home in Washington · ★4.85 · 1 bedroom · 1 bed...,This is the middle bedroom upstairs in a resto...,https://www.airbnb.com/rooms/4197,"Our area, the Eastern Market neighborhood of C...",https://a0.muscache.com/pictures/miso/Hosting-...,"Washington, District of Columbia, United States","Capitol Hill, Lincoln Park",38.88719,-76.99472,Private room in home,Private room,1,1.5 shared baths,NaN,1.0,$80.00,Hosted License: 5007242201000749
3,4529,Home in Washington · ★4.66 · 1 bedroom · 1 bed...,This is large private bedroom with plenty of...,https://www.airbnb.com/rooms/4529,Very quiet neighborhood and it is easy accessi...,https://a0.muscache.com/pictures/86072003/6709...,"Washington, District of Columbia, United States","Eastland Gardens, Kenilworth",38.90585,-76.94469,Private room in home,Private room,2,1 shared bath,NaN,1.0,$56.00,Exempt
4,4967,Home in Washington · ★4.74 · 1 bedroom · 1 bed...,"<b>The space</b><br />Hello, my name is Seveer...",https://www.airbnb.com/rooms/4967,NaN,https://a0.muscache.com/pictures/2439810/bb320...,NaN,"Ivy City, Arboretum, Trinidad, Carver Langston",38.91217,-76.99249,Private room in home,Private room,1,3 baths,NaN,1.0,"$2,500.00",NaN


In [9]:
# check data types
listings_df.dtypes

listing_id                  int64
name                       object
description                object
listing_url                object
neighborhood_overview      object
picture_url                object
neighbourhood              object
neighbourhood_cleansed     object
latitude                  float64
longitude                 float64
property_type              object
room_type                  object
accommodates                int64
bathrooms_text             object
bedrooms                  float64
beds                      float64
price                      object
license                    object
dtype: object

In [10]:
# remove $ from price column, convert to float
listings_df["price"] = (
    listings_df["price"].str.replace("$", "").str.replace(",", "").astype(float)
)

C:\Users\johbr\AppData\Local\Temp\ipykernel_12244\890666075.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_df["price"] = (


In [11]:
listings_df.dtypes

listing_id                  int64
name                       object
description                object
listing_url                object
neighborhood_overview      object
picture_url                object
neighbourhood              object
neighbourhood_cleansed     object
latitude                  float64
longitude                 float64
property_type              object
room_type                  object
accommodates                int64
bathrooms_text             object
bedrooms                  float64
beds                      float64
price                     float64
license                    object
dtype: object

In [12]:
# save listings_df to csv
listings_df.to_csv("./listings_cleaned.csv", index=False)

<h3>Messing about with hosts with max number of listings</h3>

In [13]:
# sort df by host with most listings
df["host_id"].value_counts()

host_id
39930655     216
107434423    208
294545484    132
30283594      91
46630199      91
            ... 
125854993      1
7989061        1
158352584      1
190135215      1
33736673       1
Name: count, Length: 3539, dtype: int64

In [14]:
# show listings for host with most listings
max_host_df = df[df["host_id"] == 39930655]
max_host_df

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
931,15734466,https://www.airbnb.com/rooms/15734466,20230913045152,2023-09-13,city scrape,Rental unit in Washington · ★4.67 · 2 bedrooms...,A charming 2 bedrooms 1 bathroom garden level ...,"Tree-lined streets, bold and charismatic row h...",https://a0.muscache.com/pictures/prohost-api/H...,39930655,...,4.90,4.96,4.62,Exempt,f,216,198,14,3,3.32
1040,16405079,https://www.airbnb.com/rooms/16405079,20230913045152,2023-09-13,city scrape,Home in Washington · ★4.72 · Studio · 1 bed · ...,"This fully furnished studio suite, named the L...",Dupont Circle stands out as a cosmopolitan jew...,https://a0.muscache.com/pictures/prohost-api/H...,39930655,...,4.92,4.99,4.72,Exempt,f,216,198,14,3,5.00
1301,18288362,https://www.airbnb.com/rooms/18288362,20230913045152,2023-09-13,city scrape,Home in Washington · ★4.77 · Studio · 1 bed · ...,"This fully furnished studio suite, named the V...",Dupont Circle stands out as a cosmopolitan jew...,https://a0.muscache.com/pictures/aaf31596-c61e...,39930655,...,4.88,4.98,4.71,Exempt,f,216,198,14,3,5.20
1331,18598439,https://www.airbnb.com/rooms/18598439,20230913045152,2023-09-13,city scrape,Home in Washington · ★4.77 · 1 bedroom · 1 bed...,"This fully furnished studio suite, named The C...",Dupont Circle stands out as a cosmopolitan jew...,https://a0.muscache.com/pictures/6d9113b2-f72e...,39930655,...,4.91,4.98,4.73,Hosted License: 5007242201002018,f,216,198,14,3,5.02
1361,18949151,https://www.airbnb.com/rooms/18949151,20230913045152,2023-09-13,city scrape,Boutique hotel in Washington · ★4.80 · 1 bedro...,Indulge yourself in the rich history of one of...,Dupont Circle stands out as a cosmopolitan jew...,https://a0.muscache.com/pictures/prohost-api/H...,39930655,...,4.86,4.98,4.78,Hosted License: 5007242201000611,f,216,198,14,3,5.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6663,974955832256346356,https://www.airbnb.com/rooms/974955832256346356,20230913045152,2023-09-13,city scrape,Rental unit in Washington · ★New · 1 bedroom ·...,"A 10 minute walk to The US Capitol Grounds, Th...",While Capitol Hill may conjure images of natio...,https://a0.muscache.com/pictures/prohost-api/H...,39930655,...,NaN,NaN,NaN,NaN,f,216,198,14,3,NaN
6664,974956286844795595,https://www.airbnb.com/rooms/974956286844795595,20230913045152,2023-09-13,city scrape,Rental unit in Washington · ★New · 1 bedroom ·...,Fully furnished studio apartment with one full...,While Capitol Hill may conjure images of natio...,https://a0.muscache.com/pictures/prohost-api/H...,39930655,...,NaN,NaN,NaN,NaN,f,216,198,14,3,NaN
6665,974960054830706995,https://www.airbnb.com/rooms/974960054830706995,20230913045152,2023-09-13,city scrape,Rental unit in Washington · ★New · 3 bedrooms ...,"Fabulous location near Shaw, Petworth and Howa...",Please note that it is a lively neighborhood w...,https://a0.muscache.com/pictures/prohost-api/H...,39930655,...,NaN,NaN,NaN,NaN,f,216,198,14,3,NaN
6666,974960514279898229,https://www.airbnb.com/rooms/974960514279898229,20230913045152,2023-09-13,city scrape,Rental unit in Washington · ★New · 3 bedrooms ...,"Fabulous location near Shaw, Petworth and Howa...",Please note that it is a lively neighborhood w...,https://a0.muscache.com/pictures/prohost-api/H...,39930655,...,NaN,NaN,NaN,NaN,f,216,198,14,3,NaN


In [15]:
max_host_df_cleaned = max_host_df[
    [
        "host_id",
        "id",
        "name",
        "host_name",
        "host_since",
        "host_location",
        "host_about",
        "host_response_time",
        "host_response_rate",
        "host_acceptance_rate",
        "host_is_superhost",
        "host_neighbourhood",
        "host_listings_count",
        "host_total_listings_count",
        "host_verifications",
        "host_has_profile_pic",
        "host_identity_verified",
    ]
]

max_host_df_cleaned.to_csv("./max_host_listings.csv", index=False)

max_host_df_cleaned

,host_id,id,name,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified
931,39930655,15734466,Rental unit in Washington · ★4.67 · 2 bedrooms...,Team,2015-07-29,"Washington, DC","Hi, I'm Nicole, the co-founder of Sojourn. At...",within an hour,100%,100%,f,Dupont Circle,316,377,"['email', 'phone']",t,t
1040,39930655,16405079,Home in Washington · ★4.72 · Studio · 1 bed · ...,Team,2015-07-29,"Washington, DC","Hi, I'm Nicole, the co-founder of Sojourn. At...",within an hour,100%,100%,f,Dupont Circle,316,377,"['email', 'phone']",t,t
1301,39930655,18288362,Home in Washington · ★4.77 · Studio · 1 bed · ...,Team,2015-07-29,"Washington, DC","Hi, I'm Nicole, the co-founder of Sojourn. At...",within an hour,100%,100%,f,Dupont Circle,316,377,"['email', 'phone']",t,t
1331,39930655,18598439,Home in Washington · ★4.77 · 1 bedroom · 1 bed...,Team,2015-07-29,"Washington, DC","Hi, I'm Nicole, the co-founder of Sojourn. At...",within an hour,100%,100%,f,Dupont Circle,316,377,"['email', 'phone']",t,t
1361,39930655,18949151,Boutique hotel in Washington · ★4.80 · 1 bedro...,Team,2015-07-29,"Washington, DC","Hi, I'm Nicole, the co-founder of Sojourn. At...",within an hour,100%,100%,f,Dupont Circle,316,377,"['email', 'phone']",t,t
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6663,39930655,974955832256346356,Rental unit in Washington · ★New · 1 bedroom ·...,Team,2015-07-29,"Washington, DC","Hi, I'm Nicole, the co-founder of Sojourn. At...",within an hour,100%,100%,f,Dupont Circle,316,377,"['email', 'phone']",t,t
6664,39930655,974956286844795595,Rental unit in Washington · ★New · 1 bedroom ·...,Team,2015-07-29,"Washington, DC","Hi, I'm Nicole, the co-founder of Sojourn. At...",within an hour,100%,100%,f,Dupont Circle,316,377,"['email', 'phone']",t,t
6665,39930655,974960054830706995,Rental unit in Washington · ★New · 3 bedrooms ...,Team,2015-07-29,"Washington, DC","Hi, I'm Nicole, the co-founder of Sojourn. At...",within an hour,100%,100%,f,Dupont Circle,316,377,"['email', 'phone']",t,t
6666,39930655,974960514279898229,Rental unit in Washington · ★New · 3 bedrooms ...,Team,2015-07-29,"Washington, DC","Hi, I'm Nicole, the co-founder of Sojourn. At...",within an hour,100%,100%,f,Dupont Circle,316,377,"['email', 'phone']",t,t


In [16]:
host_most = max_host_df_cleaned["host_name"].unique()
host_most

In [17]:
# print top 10 hosts with most listings, name and number of listings
top_hosts = df["host_id"].value_counts().head(10)
top_hosts

host_id
39930655     216
107434423    208
294545484    132
30283594      91
46630199      91
511932461     67
7650266       60
122382567     47
9419684       46
95459395      44
Name: count, dtype: int64

In [18]:
top_hosts.index

Index([ 39930655, 107434423, 294545484,  30283594,  46630199, 511932461,
         7650266, 122382567,   9419684,  95459395],
      dtype='int64', name='host_id')

In [19]:
top_hosts_df = df[df["host_id"].isin(top_hosts.index)]
top_hosts_df

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
149,1864715,https://www.airbnb.com/rooms/1864715,20230913045152,2023-09-13,previous scrape,Serviced apartment in Washington · ★4.88 · 1 b...,Samuel W. Woodward’s early 20th century archit...,This beautiful building provides comforts and ...,https://a0.muscache.com/pictures/prohost-api/H...,9419684,...,4.44,5.00,4.63,NaN,t,46,46,0,0,0.14
151,1867513,https://www.airbnb.com/rooms/1867513,20230913045152,2023-09-13,previous scrape,Serviced apartment in Washington · ★4.76 · 2 b...,Samuel W. Woodward’s early 20th century archit...,This beautiful building provides comforts and ...,https://a0.muscache.com/pictures/prohost-api/H...,9419684,...,4.48,5.00,4.72,NaN,t,46,46,0,0,0.22
180,2371799,https://www.airbnb.com/rooms/2371799,20230913045152,2023-09-13,previous scrape,Serviced apartment in Washington · ★4.79 · 2 b...,Samuel W. Woodward’s early 20th century archit...,This beautiful building provides comforts and ...,https://a0.muscache.com/pictures/prohost-api/H...,9419684,...,4.64,4.93,4.71,NaN,t,46,46,0,0,0.12
260,4092017,https://www.airbnb.com/rooms/4092017,20230913045152,2023-09-13,previous scrape,Serviced apartment in Washington · ★4.67 · 1 b...,Samuel W. Woodward’s early 20th century archit...,This beautiful building provides comforts and ...,https://a0.muscache.com/pictures/prohost-api/H...,9419684,...,5.00,5.00,4.67,NaN,t,46,46,0,0,0.03
277,4447266,https://www.airbnb.com/rooms/4447266,20230913045152,2023-09-13,previous scrape,Serviced apartment in Washington · ★4.90 · 1 b...,Samuel W. Woodward’s early 20th century archit...,This beautiful building provides comforts and ...,https://a0.muscache.com/pictures/prohost-api/H...,9419684,...,4.44,5.00,4.70,NaN,t,46,46,0,0,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6667,974960700640417299,https://www.airbnb.com/rooms/974960700640417299,20230913045152,2023-09-13,city scrape,Rental unit in Washington · ★New · 2 bedrooms ...,"Fabulous location near Shaw, Petworth and Howa...",Please note that it is a lively neighborhood w...,https://a0.muscache.com/pictures/prohost-api/H...,39930655,...,NaN,NaN,NaN,NaN,f,216,198,14,3,NaN
6680,975326861225794486,https://www.airbnb.com/rooms/975326861225794486,20230913045152,2023-09-13,city scrape,Serviced apartment in Washington · ★New · 2 be...,Experience the New York City lifestyle in the ...,Tribeca is the perfect mix of Suburban and Cit...,https://a0.muscache.com/pictures/prohost-api/H...,30283594,...,NaN,NaN,NaN,Exempt,t,91,91,0,0,NaN
6681,975342105048109613,https://www.airbnb.com/rooms/975342105048109613,20230913045152,2023-09-13,city scrape,Serviced apartment in Washington · ★New · 2 be...,Experience the New York City lifestyle in the ...,Tribeca is the perfect mix of Suburban and Cit...,https://a0.muscache.com/pictures/prohost-api/H...,30283594,...,NaN,NaN,NaN,Exempt,t,91,91,0,0,NaN
6696,977375107968316358,https://www.airbnb.com/rooms/977375107968316358,20230913045152,2023-09-13,city scrape,Rental unit in Washington · ★New · 1 bedroom ·...,Welcome to our cozy 1BR/1BA garden-level renta...,Mount Pleasant`s rich historical connection to...,https://a0.muscache.com/pictures/prohost-api/H...,46630199,...,NaN,NaN,NaN,Hosted License: 5007242201002847,f,91,91,0,0,NaN


In [20]:
top_hosts_summary = (
    top_hosts_df.groupby(["host_id", "host_name"])
    .size()
    .reset_index(name="listings_count")
)
top_hosts_summary

,host_id,host_name,listings_count
0,7650266,Stay Attache,60
1,9419684,Churchill,46
2,30283594,Global Luxury Suites,91
3,39930655,Team,216
4,46630199,Home Sweet City,91
5,95459395,Bluebird,44
6,107434423,Blueground,208
7,122382567,Evolve,47
8,294545484,June,132
9,511932461,Zeus DC,67


In [21]:
top_hosts_summary = top_hosts_summary.sort_values(by="listings_count", ascending=False)
top_hosts_summary

,host_id,host_name,listings_count
3,39930655,Team,216
6,107434423,Blueground,208
8,294545484,June,132
2,30283594,Global Luxury Suites,91
4,46630199,Home Sweet City,91
9,511932461,Zeus DC,67
0,7650266,Stay Attache,60
7,122382567,Evolve,47
1,9419684,Churchill,46
5,95459395,Bluebird,44


<h3>Weird. The host with the most listings is Sojourn. Team Soujourn? The web scrape lost some info</h3>

<h2>Recreate original hosts df</h2>

In [22]:
# create hosts df
hosts_df = df[
    [
        "host_id",
        "host_url",
        "host_name",
        "host_since",
        "host_location",
        "host_about",
        "host_response_time",
        "host_response_rate",
        "host_acceptance_rate",
        "host_is_superhost",
        "host_thumbnail_url",
        "host_picture_url",
        "host_neighbourhood",
        "host_listings_count",
        "host_total_listings_count",
        "host_verifications",
        "host_has_profile_pic",
        "host_identity_verified",
    ]
]
hosts_df.head()

,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified
0,4645,https://www.airbnb.com/users/show/4645,Vita,2008-11-26,"Washington D.C., DC","I am a literary scholar, teacher, poet, vegan ...",within an hour,100%,100%,t,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,Anacostia,1,4,"['email', 'phone', 'work_email']",t,t
1,5059,https://www.airbnb.com/users/show/5059,Vasa,2008-12-12,"Washington, DC",I travel often and always try to immerse mysel...,within an hour,100%,92%,t,https://a0.muscache.com/im/pictures/user/8ec69...,https://a0.muscache.com/im/pictures/user/8ec69...,Eckington,5,5,"['email', 'phone']",t,t
2,5061,https://www.airbnb.com/users/show/5061,Sandra,2008-12-12,"Washington D.C., DC",I’m a California native who came to work for t...,within an hour,100%,100%,t,https://a0.muscache.com/im/pictures/user/93c77...,https://a0.muscache.com/im/pictures/user/93c77...,Capitol Hill,2,2,"['email', 'phone']",t,t
3,5803,https://www.airbnb.com/users/show/5803,Bertina,2008-12-30,"Washington, DC",I am a retried teacher. I enjoy walking and wa...,NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/0050a...,https://a0.muscache.com/im/pictures/user/0050a...,Eastland Gardens,3,4,"['email', 'phone']",t,t
4,7086,https://www.airbnb.com/users/show/7086,Edward,2009-01-26,NaN,"I am fun, honest and very easy going and trave...",within a few hours,100%,95%,t,https://a0.muscache.com/im/pictures/user/6efb4...,https://a0.muscache.com/im/pictures/user/6efb4...,Ivy City,3,5,"['email', 'phone']",t,t


In [23]:
# check data types
hosts_df.dtypes

host_id                       int64
host_url                     object
host_name                    object
host_since                   object
host_location                object
host_about                   object
host_response_time           object
host_response_rate           object
host_acceptance_rate         object
host_is_superhost            object
host_thumbnail_url           object
host_picture_url             object
host_neighbourhood           object
host_listings_count           int64
host_total_listings_count     int64
host_verifications           object
host_has_profile_pic         object
host_identity_verified       object
dtype: object

In [24]:
# convert host_since to datetime
hosts_df["host_since"] = pd.to_datetime(hosts_df["host_since"])

C:\Users\johbr\AppData\Local\Temp\ipykernel_12244\2881374419.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hosts_df["host_since"] = pd.to_datetime(hosts_df["host_since"])


In [25]:
hosts_df.dtypes

host_id                               int64
host_url                             object
host_name                            object
host_since                   datetime64[ns]
host_location                        object
host_about                           object
host_response_time                   object
host_response_rate                   object
host_acceptance_rate                 object
host_is_superhost                    object
host_thumbnail_url                   object
host_picture_url                     object
host_neighbourhood                   object
host_listings_count                   int64
host_total_listings_count             int64
host_verifications                   object
host_has_profile_pic                 object
host_identity_verified               object
dtype: object

In [26]:
hosts_df["host_is_superhost"].value_counts()
hosts_df["host_is_superhost"].isna().sum()

66

In [27]:
print(hosts_df["host_has_profile_pic"].value_counts())
print(hosts_df["host_has_profile_pic"].isna().sum())
print(hosts_df["host_identity_verified"].value_counts())
print(hosts_df["host_identity_verified"].isna().sum())

host_has_profile_pic
t    6617
f      88
Name: count, dtype: int64
0
host_identity_verified
t    5840
f     865
Name: count, dtype: int64
0


In [28]:
# convert host_is_superhost, host_has_profile_pic, host_identity_verified to boolean
columns_to_bool = [
    "host_is_superhost",
    "host_has_profile_pic",
    "host_identity_verified",
]
for column in columns_to_bool:
    hosts_df[column] = hosts_df[column].map({"t": True, "f": False}).astype("boolean")


hosts_df.dtypes

C:\Users\johbr\AppData\Local\Temp\ipykernel_12244\2993338222.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hosts_df[column] = hosts_df[column].map({"t": True, "f": False}).astype("boolean")
C:\Users\johbr\AppData\Local\Temp\ipykernel_12244\2993338222.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hosts_df[column] = hosts_df[column].map({"t": True, "f": False}).astype("boolean")
C:\Users\johbr\AppData\Local\Temp\ipykernel_12244\2993338222.py:8: SettingWithCopyWarning: 
A value is trying to be set 

host_id                               int64
host_url                             object
host_name                            object
host_since                   datetime64[ns]
host_location                        object
host_about                           object
host_response_time                   object
host_response_rate                   object
host_acceptance_rate                 object
host_is_superhost                   boolean
host_thumbnail_url                   object
host_picture_url                     object
host_neighbourhood                   object
host_listings_count                   int64
host_total_listings_count             int64
host_verifications                   object
host_has_profile_pic                boolean
host_identity_verified              boolean
dtype: object

In [29]:
hosts_df.head()

,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified
0,4645,https://www.airbnb.com/users/show/4645,Vita,2008-11-26,"Washington D.C., DC","I am a literary scholar, teacher, poet, vegan ...",within an hour,100%,100%,True,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,Anacostia,1,4,"['email', 'phone', 'work_email']",True,True
1,5059,https://www.airbnb.com/users/show/5059,Vasa,2008-12-12,"Washington, DC",I travel often and always try to immerse mysel...,within an hour,100%,92%,True,https://a0.muscache.com/im/pictures/user/8ec69...,https://a0.muscache.com/im/pictures/user/8ec69...,Eckington,5,5,"['email', 'phone']",True,True
2,5061,https://www.airbnb.com/users/show/5061,Sandra,2008-12-12,"Washington D.C., DC",I’m a California native who came to work for t...,within an hour,100%,100%,True,https://a0.muscache.com/im/pictures/user/93c77...,https://a0.muscache.com/im/pictures/user/93c77...,Capitol Hill,2,2,"['email', 'phone']",True,True
3,5803,https://www.airbnb.com/users/show/5803,Bertina,2008-12-30,"Washington, DC",I am a retried teacher. I enjoy walking and wa...,NaN,NaN,NaN,False,https://a0.muscache.com/im/pictures/user/0050a...,https://a0.muscache.com/im/pictures/user/0050a...,Eastland Gardens,3,4,"['email', 'phone']",True,True
4,7086,https://www.airbnb.com/users/show/7086,Edward,2009-01-26,NaN,"I am fun, honest and very easy going and trave...",within a few hours,100%,95%,True,https://a0.muscache.com/im/pictures/user/6efb4...,https://a0.muscache.com/im/pictures/user/6efb4...,Ivy City,3,5,"['email', 'phone']",True,True


In [37]:
# check value_counts
hosts_df["host_id"].value_counts()

host_id
39930655     216
107434423    208
294545484    132
30283594      91
46630199      91
            ... 
125854993      1
7989061        1
158352584      1
190135215      1
33736673       1
Name: count, Length: 3539, dtype: int64

In [40]:
# drop duplicate rows
hosts_df = hosts_df.drop_duplicates(subset=["host_id"])
hosts_df["host_id"].value_counts()

host_id
4645         1
35328672     1
210785866    1
13242357     1
349251028    1
            ..
157832931    1
51590466     1
155256037    1
158129211    1
33736673     1
Name: count, Length: 3539, dtype: int64

In [41]:
# save to csv
hosts_df.to_csv("./hosts_cleaned.csv", index=False)

In [31]:
df.dtypes

id                                                int64
listing_url                                      object
scrape_id                                         int64
last_scraped                                     object
source                                           object
                                                 ...   
calculated_host_listings_count                    int64
calculated_host_listings_count_entire_homes       int64
calculated_host_listings_count_private_rooms      int64
calculated_host_listings_count_shared_rooms       int64
reviews_per_month                               float64
Length: 75, dtype: object

In [32]:
listings_df.dtypes

listing_id                  int64
name                       object
description                object
listing_url                object
neighborhood_overview      object
picture_url                object
neighbourhood              object
neighbourhood_cleansed     object
latitude                  float64
longitude                 float64
property_type              object
room_type                  object
accommodates                int64
bathrooms_text             object
bedrooms                  float64
beds                      float64
price                     float64
license                    object
dtype: object

In [33]:
neighbourhood_df = df[["neighbourhood", "neighbourhood_cleansed", "neighbourhood_group_cleansed"]]
neighbourhood_df.head()

,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed
0,"Washington, District of Columbia, United States",Historic Anacostia,NaN
1,"Washington, District of Columbia, United States","Edgewood, Bloomingdale, Truxton Circle, Eckington",NaN
2,"Washington, District of Columbia, United States","Capitol Hill, Lincoln Park",NaN
3,"Washington, District of Columbia, United States","Eastland Gardens, Kenilworth",NaN
4,NaN,"Ivy City, Arboretum, Trinidad, Carver Langston",NaN


In [34]:
df[["neighbourhood", "neighbourhood_cleansed", "neighbourhood_group_cleansed"]]


,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed
0,"Washington, District of Columbia, United States",Historic Anacostia,NaN
1,"Washington, District of Columbia, United States","Edgewood, Bloomingdale, Truxton Circle, Eckington",NaN
2,"Washington, District of Columbia, United States","Capitol Hill, Lincoln Park",NaN
3,"Washington, District of Columbia, United States","Eastland Gardens, Kenilworth",NaN
4,NaN,"Ivy City, Arboretum, Trinidad, Carver Langston",NaN
...,...,...,...
6700,"Washington, District of Columbia, United States","Capitol Hill, Lincoln Park",NaN
6701,NaN,"Capitol Hill, Lincoln Park",NaN
6702,NaN,"Georgetown, Burleith/Hillandale",NaN
6703,NaN,"Edgewood, Bloomingdale, Truxton Circle, Eckington",NaN


In [35]:
print(f'Number of neighbourhood NaN\'s: {df["neighbourhood"].isna().sum()}')
print(f'Number of neighbourhood_cleansed NaN\'s: {df["neighbourhood_cleansed"].isna().sum()}')
print(f'Number of neighbourhood_cleansed: {df["neighbourhood_cleansed"].nunique()}\n')
print(f'Number of neighbourhood value counts: {df["neighbourhood"].value_counts()}\n')
df["neighbourhood_cleansed"].value_counts()


Number of neighbourhood NaN's: 2001
Number of neighbourhood_cleansed NaN's: 0
Number of neighbourhood_cleansed: 39

Number of neighbourhood value counts: neighbourhood
Washington, District of Columbia, United States                   4662
Washington, D.C., District of Columbia, United States               31
Washington, United States                                            2
Dupont Circle, Washington, District of Columbia, United States       1
Dupont Circle, District of Columbia, United States                   1
Georgetown, District of Columbia, United States                      1
Chevy Chase, Maryland, United States                                 1
 , United States                                                     1
Washington, Dc, United States                                        1
El Chorro, Departamento de Maldonado, Uruguay                        1
Alexandria, Virginia, United States                                  1
Washington DC, USA, United States                  

neighbourhood_cleansed
Union Station, Stanton Park, Kingman Park                                                            799
Capitol Hill, Lincoln Park                                                                           598
Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View                                           525
Edgewood, Bloomingdale, Truxton Circle, Eckington                                                    507
Shaw, Logan Circle                                                                                   457
Dupont Circle, Connecticut Avenue/K Street                                                           451
Brightwood Park, Crestwood, Petworth                                                                 354
Ivy City, Arboretum, Trinidad, Carver Langston                                                       276
Downtown, Chinatown, Penn Quarters, Mount Vernon Square, North Capitol Street                        261
Howard University, Le Droit Park

<h3>Neighborhood_cleansed is the only useful column - build an index for it

Have to add host_id to listings in pgadmin


In [42]:
# extract host_id and listing id
host_id_for_listings = df[["id", "host_id"]]
len(host_id_for_listings)

6705

In [43]:
# export to csv
host_id_for_listings.to_csv("./host_id_for_listings.csv", index=False)